# Анализ базы данных кино

## 1. Постановка задачи

Собрана база данных кино. В базе присутствуют столбцы с id, названия, дату релиза, описание, показы, средняя оценка, количество оценок и наличие видео. Всего собрано 9472 записи.

**Основные студии, которые могут стать заказчиками:**
- Walt Disney Company
- Базелевс
- Yellow, Black and White Group
- Paramount Pictures
- Кинокомпания СТВ

**Цели анализа:**
1. Выявить среднюю разницу оценки первой и последующих частей фильмов
2. Определить, какие слова чаще встречаются в описании фильмов с высокой оценкой
3. Найти наиболее удачные года кинопроката
4. Обнаружить закономерности дат выпуска относительно среднего успеха фильмов

Данная обработка полезна для:
- Понимания динамики успешности кинофраншиз
- Оптимизации контент-стратегии (ключевые слова для успешных фильмов)
- Планирования релизов и производства

## 2. Паспорт датасета (Data Understanding)

**Общая информация:**
- Объем датасета: 9472 записи
- Период: вероятно, современные фильмы (нужно уточнить по данным)
- Источник: агрегированные данные о фильмах

**Структура данных (предполагаемые столбцы):**

1. **id** - уникальный идентификатор фильма
   - Тип: int/string
   - Назначение: первичный ключ

2. **title** - название фильма
   - Тип: string
   - Особенности: возможны дубли для ремейков

3. **release_date** - дата релиза
   - Тип: datetime
   - Важно: преобразовать для временного анализа

4. **description** - описание фильма
   - Тип: string (текст)
   - Для NLP анализа

5. **views** - количество показов/просмотров
   - Тип: numeric
   - Показатель популярности

6. **average_rating** - средняя оценка
   - Тип: float (0-10 или 0-5)
   - Основной целевой показатель качества

7. **rating_count** - количество оценок
   - Тип: integer
   - Показатель репрезентативности оценки

8. **has_video** - наличие видео
   - Тип: boolean
   - Возможный индикатор полноты данных

**География:**
- Международные и российские студии
- Преимущественно коммерческое кино

**Потенциальные проблемы данных:**
1. Пропуски в описаниях
2. Разные системы оценок (нужна стандартизация)
3. Дубликаты фильмов (разные релизы)
4. Выбросы в показах/оценках
5. Несбалансированность по студиям

**Метрики для анализа:**
- Качество: средний рейтинг
- Популярность: количество показов/оценок
- Коммерческий успех: производные метрики

In [ ]:
# 3. Импорт библиотек
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import re
from collections import Counter

# Установка стилей
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

print("Библиотеки успешно импортированы")

In [ ]:
# 4. Загрузка данных (пример кода)
# Раскомментируйте и адаптируйте под ваш файл:
# df = pd.read_csv('movie_database.csv')
# print(f"Размер датасета: {df.shape}")
# print(f"Колонки: {df.columns.tolist()}")
# df.head()